<a href="https://colab.research.google.com/github/Soumyadip-Ghosh-18/Soumyadip-Ghosh-18/blob/main/Soumyadip_langgraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# STEP 0: Install dependencies

In [51]:
!pip install -qU google-generativeai==0.8.5 google-ai-generativelanguage==0.6.15 langgraph langchain langchain-google-genai openai

# STEP 1: Imports and secure API key input

In [52]:
import os
import getpass
# getpass used to get password for api to work & nobody can view the api.
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage



# Secure Gemini API Key input

In [53]:
os.environ['GOOGLE_API_KEY'] = getpass.getpass("Enter the Gemini API Key")

Enter the Gemini API Key··········


# STEP 2: Initialize Gemini 1.5 Flash

In [54]:
llm = ChatGoogleGenerativeAI(model = "models/gemini-1.5-flash-latest", temperature =0.5)
# temperature -> How much varities of answers are needed.

# STEP 3: Node to ask user for symptom

In [55]:
def get_symptom(state: dict) -> dict:
  symptom = input("Welcome to Apolo Hospitals: Enter your symptom.\n")
  state["symptom"] = symptom
  return state

# STEP 4: Node to classify the symptom

In [56]:
def classify_symptom(state: dict) -> dict:
  prompt = ("You are a medical assistent in some hospital and your task is to classify symptoms in one of the following catagories. \n"
  "General\n -Emergency\n -Serious \n"
  f"Symptom : {state['symptom']} \n"
  "Respond in one word: General, Emergency, Serious \n"
  "#Example : input : I have fever, Output : General"
  )
  response = llm.invoke([HumanMessage(content = prompt)])
  catagory = response.content.strip()
  print("LLM identifies the symptom as", catagory)
  state["catagory"] = catagory
  return state


# STEP 5: Router logic to route to the correct node


In [57]:
def symptom_router(state: dict) -> dict:
  cat = state['catagory'].lower()
  if cat == 'general':
    return 'general'
  elif cat == 'emergency':
    return 'emergency'
  elif cat == 'serious':
    return 'serious'
  else:
    return 'critical'

# STEP 6: Category-specific response nodes


In [58]:
def general_node(state: dict) -> dict:
  state['answer'] = f"'{state['symptom']}': Seems like a General case: reffering to General Ward, consult a doctor in there."
  return state
def emergency_node(state: dict) -> dict:
  state['answer'] = f"'{state['symptom']}': It is an Emergency case: Urgent treatment needed: reffering to Emergency Ward."
  return state
def serious_node(state: dict) -> dict:
  state['answer'] = f"'{state['symptom']}': Seems like it is a Serious case: admiting you, booking a doctor specifically for you."
  return state
def critical_node(state: dict) -> dict:
  state['answer'] = f"'{state['symptom']}': It is a Critical case: get admission as early as possible, now I am booking a meeting with specilized doctor."
  return state

# STEP 7: Build LangGraph


In [59]:
builder = StateGraph(dict)

#define the nodes
builder.set_entry_point("get_symptom")
builder.add_node("get_symptom", get_symptom)
builder.add_node("classify", classify_symptom)
builder.add_node("general", general_node)
builder.add_node("emergency", emergency_node)
builder.add_node("serious", serious_node)
builder.add_node("critical", critical_node)

builder.add_edge("get_symptom", "classify")
builder.add_conditional_edges("classify", symptom_router, {
    "general": "general",
    "emergency": "emergency",
    "serious": "serious"
})

builder.add_edge("general", END)
builder.add_edge("emergency", END)
builder.add_edge("serious", END)
builder.add_edge("critical", END)


# STEP 8: Compile and invoke the graph


In [60]:
graph = builder.compile()

In [61]:
final_state = graph.invoke({})
print("Final Output \n")
print(final_state['answer'])

Welcome to Apolo Hospitals: Enter your symptom.
I am feeling very little pain in my full body, and facing lower energy levels while doing any work.
LLM identifies the symptom as General
Final Output 

'I am feeling very little pain in my full body, and facing lower energy levels while doing any work.': Seems like a General case: reffering to General Ward, consult a doctor in there.
